In [2]:
import tensorflow as tf

print(tf.version.VERSION)

2.4.1


In [14]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X,Y = datasets.load_iris(return_X_y = True)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2)

clf = tf.estimator.LinearClassifier()

clf.train(X_train)

TypeError: ignored